# Labeling Sentimen Komentar YouTube dengan Gemini (Semi-Auto)

Notebook ini digunakan untuk melakukan **pseudo-labeling** sentimen komentar YouTube
menggunakan **Gemini API**, kemudian hasil label akan direview dan diperbaiki secara manual.

Alur pada notebook ini:
1. Load dataset raw hasil scrapping (`raw_comments.csv`)
2. (Opsional) Memilih subset komentar yang akan dilabeli
3. Memanggil Gemini API untuk memberi label awal (positive / neutral / negative)
4. Menyimpan hasil ke file `pseudo_labels.csv`
5. File `pseudo_labels.csv` akan direview dan diperbaiki menjadi `labeled_data.csv`

**Catatan penting:**
- Labeling dilakukan pada **teks raw**, bukan teks yang sudah dipreprocessing.
- Hasil Gemini hanya sebagai label awal (*model-assisted labeling*), 
  bukan label final tanpa koreksi manusia.


## Import Library

In [1]:
import os
import time
import pandas as pd
from tqdm.auto import tqdm

from dotenv import load_dotenv
import google.generativeai as genai

c:\Users\hafizh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Konfigurasi Gemini API

In [ ]:
from dotenv import load_dotenv
import google.generativeai as genai
import os

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if api_key is None:
    raise ValueError("GEMINI_API_KEY tidak ditemukan di .env")

genai.configure(api_key=api_key)

MODEL_NAME = "models/gemini-2.0-flash-lite"
model = genai.GenerativeModel(MODEL_NAME)
print("Pakai model:", MODEL_NAME)

COMMENTS_PER_BATCH = 200   # berapa komentar per batch file (misal 200)
DELAY_SECONDS      = 20.0  # delay antar request ke Gemini
BACKOFF_SECONDS    = 40.0  # kalau kena rate limit, tidur lebih lama dulu

# 4. Path data
RAW_PATH        = "data/raw_dataset_whoosh.csv"
BATCH_DIR       = "data"
ALL_OUTPUT_PATH = os.path.join(BATCH_DIR, "pseudo_labels_all.csv")

os.makedirs(BATCH_DIR, exist_ok=True)

Pakai model: models/gemini-2.0-flash


## Import Dataset

Dataset yang digunakan adalah hasil scrapping komentar YouTube : `data/raw_comments.csv`.

Kolom penting:
- `comment` : isi komentar asli (raw)
- `video_id`, `video_title` : informasi video (metadata)

In [3]:
df = pd.read_csv(r"D:\Arsip Hafizh Fadhl Muhammad\Project\project-sentimen-analisis-datmin\data\raw_dataset_whoosh.csv")

In [4]:
print("Jumlah baris:", len(df))

Jumlah baris: 1000


In [5]:
df.head()

,video_id,video_title,comment_id,author,comment,likes,published_at
0,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzlM3ejBnN9PKqB2sB4AaABAg,@MA_Alpha-l4q,SUDAH JELAS GENG SOLO YANG HARUS BERTANGGUNG J...,1,2025-12-01T06:59:37Z
1,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzcxKsZ3V_n222CvEF4AaABAg,@nurhasanahssi2114,"Jokowi, Luhut, kroni2 yg harus bertanggungjaw...",0,2025-11-30T01:18:59Z
2,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,Ugx1UtqOkCAflakGpoh4AaABAg,@omsimon-k6k,Yg ditangkap gorengan yg makan duduk manis,0,2025-11-28T15:56:25Z
3,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,Ugz7lA1xVXYLNUEGjEp4AaABAg,@mohammadharriszulfika8486,buat bayar hutang whossh jual saja Aset tentar...,0,2025-11-28T14:11:29Z
4,1_Xrj0mb7K4,Bedah KEGILAAN Project Whoosh,UgzeBmAExGNOkXuCQO94AaABAg,@isaansyori8749,Pantas saja ngotot bgt lanjut 3 periode ternya...,1,2025-11-28T13:19:57Z


## Definisi Skema Label & Prompt Gemini

Skema label sentimen yang digunakan:

- `positive` : komentar bernada mendukung, apresiasi, atau optimis
- `neutral`  : komentar informatif, bertanya, atau tidak jelas emosinya
- `negative` : komentar bernada marah, sinis, kecewa, atau menghina

Gemini akan diminta:
- membalas **hanya** dengan salah satu dari: `positive`, `neutral`, `negative`
- tanpa penjelasan panjang (untuk menghemat token)


In [ ]:
SYSTEM_PROMPT = """
Kamu adalah model analisis sentimen untuk komentar YouTube dalam bahasa Indonesia.

Tugasmu:
- Baca komentar berikut.
- Tentukan sentimennya: positive, neutral, atau negative.

Aturan:
- Balas HANYA dengan salah satu kata ini saja (huruf kecil semua):
  - positive
  - neutral
  - negative
- Jangan menambahkan teks lain, jangan menambahkan penjelasan.
"""

VALID_LABELS = {"positive", "neutral", "negative"}

## Fungsi untuk Meminta Label dari Gemini

Fungsi `get_sentiment_from_gemini(text)` akan:
- mengirim komentar ke Gemini
- menerima jawaban label (positive / neutral / negative)
- jika respons tidak valid, dikembalikan sebagai `None`


In [ ]:
def get_sentiment_from_gemini(text: str) -> str | None:
    """
    Meminta label sentimen untuk satu komentar ke Gemini.
    - Pakai DELAY_SECONDS setelah request berhasil.
    - Kalau kena rate limit (429 / quota), tidur BACKOFF_SECONDS dan ulang.
    """
    text = str(text).strip()
    if not text:
        return None

    while True:
        try:
            prompt = f"{SYSTEM_PROMPT}\n\nKomentar:\n\"\"\"{text}\"\"\""
            response = model.generate_content(prompt)
            raw_output = (response.text or "").strip().lower()
            label = raw_output.split()[0] if raw_output else ""

            if label not in VALID_LABELS:
                print(f"[WARNING] Label tidak valid: '{raw_output}' -> None")
                label = None

            # Delay normal antar request untuk jaga rate limit
            time.sleep(DELAY_SECONDS)
            return label

        except Exception as e:
            err = str(e)
            # Rate limit / quota
            if "429" in err or "quota" in err or "Resource exhausted" in err:
                print(f"[INFO] Kena rate limit, tidur dulu {BACKOFF_SECONDS} detik...")
                time.sleep(BACKOFF_SECONDS)
                # kemudian while True akan mengulang request
                continue
            else:
                print("[ERROR] Gagal panggil Gemini:", e)
                return None

Dataset besar, ambil subset 400 komentar untuk labeling.


400

In [ ]:
def label_one_batch(df_batch: pd.DataFrame, batch_idx: int) -> pd.DataFrame:
    """
    Melabeli satu batch DataFrame (df_batch) dan menyimpannya ke file:
    data/pseudo_labels_batch_{batch_idx}.csv
    """
    df_batch = df_batch.copy().reset_index(drop=True)
    df_batch["sentiment"] = None

    print(f"\n=== Mulai batch {batch_idx} (jumlah komentar: {len(df_batch)}) ===")
    for i in tqdm(range(len(df_batch)), desc=f"Batch {batch_idx}"):
        text = df_batch.loc[i, "comment"]
        df_batch.loc[i, "sentiment"] = get_sentiment_from_gemini(text)

    # Simpan batch
    batch_path = os.path.join(BATCH_DIR, f"pseudo_labels_batch_{batch_idx}.csv")
    df_batch.to_csv(batch_path, index=False, encoding="utf-8")
    print(f"[INFO] Batch {batch_idx} selesai dan disimpan di: {batch_path}")

    return df_batch

In [ ]:
# Hitung jumlah batch
n_rows = len(df)
n_batches = math.ceil(n_rows / COMMENTS_PER_BATCH)
print(f"Total baris: {n_rows}, COMMENTS_PER_BATCH: {COMMENTS_PER_BATCH}, n_batches: {n_batches}")

all_batches = []  # untuk menampung DataFrame tiap batch (kalau kamu mau langsung gabung)

for batch_idx in range(1, n_batches + 1):
    start = (batch_idx - 1) * COMMENTS_PER_BATCH
    end   = min(batch_idx * COMMENTS_PER_BATCH, n_rows)

    df_batch = df.iloc[start:end]
    labeled_batch = label_one_batch(df_batch, batch_idx)
    all_batches.append(labeled_batch)

print("\n=== Semua batch yang dijalankan sudah selesai. Sekarang gabung... ===")

# Cara 1: gabung dari DF yang baru saja diproses (kalau semua batch selesai dalam 1x run)
df_all = pd.concat(all_batches, ignore_index=True)

# Cara 2 (lebih aman): gabung dari semua file batch di folder data/
#   → ini berguna kalau misalnya notebook mati di tengah jalan, tapi beberapa batch sudah tersimpan.
# import glob
# batch_files = sorted(glob.glob(os.path.join(BATCH_DIR, "pseudo_labels_batch_*.csv")))
# df_all = pd.concat([pd.read_csv(f) for f in batch_files], ignore_index=True)

# Hilangkan duplikat berdasarkan comment_id (kalau kolom ini ada)
if "comment_id" in df_all.columns:
    before = len(df_all)
    df_all = df_all.drop_duplicates(subset="comment_id").reset_index(drop=True)
    after = len(df_all)
    print(f"Menghapus duplikat comment_id: {before - after} baris duplikat dihapus.")

# Simpan file gabungan
df_all.to_csv(ALL_OUTPUT_PATH, index=False, encoding="utf-8")
print(f"[INFO] Semua batch digabung dan disimpan di: {ALL_OUTPUT_PATH}")

df_all.head()